# Advanced RAG on Hugging Face documentation using LangChain

Mainly followed: https://huggingface.co/learn/cookbook/advanced_rag#load-your-knowledge-base

# Imports

In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from transformers import AutoTokenizer, pipeline
from sentence_transformers import SentenceTransformer

import re
import chromadb
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader, PyPDFDirectoryLoader

FILE_PATH = 'data/Arbetsmiljö/Policy för arbetsmiljö 2022-2024.PDF'

MODEL_NAME_KBLAB = 'KBLab/sentence-bert-swedish-cased'
MODEL_NAME_KB = 'KB/bert-base-swedish-cased'
MODEL_NAME_INTFLOAT = 'intfloat/multilingual-e5-large-instruct'

c:\Users\Wendy\Chalmers_Summer\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Document Loading

In [2]:
# load pdf document. Use PyPDFDirectoryLoader for loading files in directory.
loader = PyPDFLoader(FILE_PATH)
documents = loader.load()
documents[:2]

[Document(page_content='STYRDOKUMENT: Chalmers Arbetsmiljöpolicy, Dnr C 2021-1894. Beslut fattat av rektor 2022-01-17.Dokumentets metadata:\nBeslut av:\nRektorTyp av styrdokument:\nPolicyDiarienummer:\nC 2021-1894\nDatum för\nbeslut:\n2022-01-18Handläggare:\nSamuel FröjmarkDokumentstruktur:\nD1.3\nDokumentet\ngäller från och\nmed:\n2022-01-01Avdelning/motsvarande som ansvarar för\natt dokumentet skapas och/eller\nrevideras:\nHR-avdelningenDokumentet\nreviderat,\ndatum:Versionsnummer:\nDokumentet\ngäller till och\nmed:\n2024-12-31Dokumentet ersätter tidigare beslut:\nC 2018-1491Dokumentet genomgånget utan\nändring, datum:\nArbetsmiljöpolicy för Chalmers tekniska högskola AB\nStyrdokument vid Chalmers\nArbetsmiljön i Chalmers verksamhet ska vara sådan att inga medarbetare eller studenter riskerar att\ndrabbas av ohälsa eller komma till skada på grund av sin arbetsmiljö/studiemiljö. Arbetsmiljön ska\nbidra till trivsel och utveckling både yrkesmässigt och individuellt. Alla beslut som fat

# Document Transformers

In [3]:
import matplotlib.pyplot as plt
import pandas as pd

print(f"Model's maximum sequence length: {SentenceTransformer(MODEL_NAME_KBLAB).max_seq_length}")

# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME_KBLAB)
# lengths = [len(tokenizer.encode(doc.page_content)) for doc in documents]

# # Plot the distribution of document lengths, counted as the number of tokens
# fig = pd.Series(lengths).hist()
# plt.title("Distribution of document lengths in the knowledge base (in count of tokens)")
# plt.show()

c:\Users\Wendy\Chalmers_Summer\.venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Model's maximum sequence length: 384


In [4]:
# We use a hierarchical list of separators specifically tailored for splitting Markdown documents
# This list is taken from LangChain's MarkdownTextSplitter class
MARKDOWN_SEPARATORS = [
    "\n\n\n\n",
    "\n\n\n",
    "\n\n",
    "\n",
    ".",
    ",",
    " ",
    "",
]

def split_documents(chunk_size, knowledge_base, tokenizer_name):
    """
    Split documents into chunks of maximum size `chunk_size` tokens and return a list of documents.
    """
    text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
        tokenizer=AutoTokenizer.from_pretrained(tokenizer_name),
        chunk_size=chunk_size,
        chunk_overlap=chunk_size // 10,
        add_start_index=True,
        strip_whitespace=True,
        separators=MARKDOWN_SEPARATORS,
    )

    docs_processed = []
    for doc in knowledge_base:
        docs_processed += text_splitter.split_documents([doc])

    # Remove duplicates
    unique_texts = {}
    docs_processed_unique = []
    for doc in docs_processed:
        if doc.page_content not in unique_texts:
            unique_texts[doc.page_content] = True
            docs_processed_unique.append(doc)

    return docs_processed_unique

# Remove all whitespaces between newlines e.g. \n \n \n \n --> \n\n\n\n
for doc in documents:
    doc.page_content = re.sub('(?<=\\n) (?=\\n)', '', doc.page_content)

docs = split_documents(
    384,  # We choose a chunk size adapted to our model
    documents,
    tokenizer_name=MODEL_NAME_KBLAB,
)
docs[0].page_content

'STYRDOKUMENT: Chalmers Arbetsmiljöpolicy, Dnr C 2021-1894. Beslut fattat av rektor 2022-01-17.Dokumentets metadata:\nBeslut av:\nRektorTyp av styrdokument:\nPolicyDiarienummer:\nC 2021-1894\nDatum för\nbeslut:\n2022-01-18Handläggare:\nSamuel FröjmarkDokumentstruktur:\nD1.3\nDokumentet\ngäller från och\nmed:\n2022-01-01Avdelning/motsvarande som ansvarar för\natt dokumentet skapas och/eller\nrevideras:\nHR-avdelningenDokumentet\nreviderat,\ndatum:Versionsnummer:\nDokumentet\ngäller till och\nmed:\n2024-12-31Dokumentet ersätter tidigare beslut:\nC 2018-1491Dokumentet genomgånget utan\nändring, datum:\nArbetsmiljöpolicy för Chalmers tekniska högskola AB\nStyrdokument vid Chalmers\nArbetsmiljön i Chalmers verksamhet ska vara sådan att inga medarbetare eller studenter riskerar att\ndrabbas av ohälsa eller komma till skada på grund av sin arbetsmiljö/studiemiljö. Arbetsmiljön ska\nbidra till trivsel och utveckling både yrkesmässigt och individuellt. Alla beslut som fattas och alla\naktivitet

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME_KBLAB)
# lengths = [len(tokenizer.encode(doc.page_content)) for doc in docs]
# fig = pd.Series(lengths).hist()
# plt.title("Distribution of document lengths in the knowledge base (in count of tokens)")
# plt.show()

# Text Embedding & Vector Stores

In [6]:
# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=MODEL_NAME_KBLAB, # Provide the pre-trained model's path
    model_kwargs={'device':'cpu'}, # Pass the model configuration options
    encode_kwargs={'normalize_embeddings': True} # Set `True` for cosine similarity
)

# Initialize Chroma DB
chroma_client = chromadb.Client()

# switch `create_collection` to `get_or_create_collection` to avoid creating a new collection every time
collection = chroma_client.get_or_create_collection(name="huggingface_collection")

# load it into Chroma
db = Chroma.from_documents(
    documents=docs,
    embedding=embeddings,
    collection_name='huggingface_collection',
    client=chroma_client,
)
print(f"Added {len(docs)} chunks to ChromaDB")

Added 3 chunks to ChromaDB


# Preparing the LLM Model


In [21]:
from transformers import AutoModelForCausalLM
import torch
import os

# model_name = 'KBLab/bart-base-swedish-cased'
model_name = 'AI-Sweden-Models/gpt-sw3-356m-instruct'
access_token = os.getenv('HF_TOKEN')

# Initialize Tokenizer & Model
tokenizer = AutoTokenizer.from_pretrained(model_name, token=access_token)
model = AutoModelForCausalLM.from_pretrained(model_name, token=access_token)
device = "cuda:0" if torch.cuda.is_available() else "cpu"

text_generation_pipeline = pipeline(
    'text-generation',
    model=model,
    tokenizer=tokenizer,
    device=device,
    do_sample=True, 
    temperature=0.6, 
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=500,
)

In [22]:
prompt_in_chat_format = [
    {
        "role": "system",
        "content": """Using the information contained in the context,
give a comprehensive answer to the question.
Respond only to the question asked, response should be concise and relevant to the question.
Provide the number of the source document when relevant.
If the answer cannot be deduced from the context, do not give an answer.""",
    },
    {
        "role": "user",
        "content": """Context:
{context}
---
Now here is the question you need to answer.

Question: {question}""",
    },
]
RAG_PROMPT_TEMPLATE = tokenizer.apply_chat_template(
    prompt_in_chat_format, tokenize=False, add_generation_prompt=True
)
print(RAG_PROMPT_TEMPLATE)

<|endoftext|><s>Bot: Using the information contained in the context,
give a comprehensive answer to the question.
Respond only to the question asked, response should be concise and relevant to the question.
Provide the number of the source document when relevant.
If the answer cannot be deduced from the context, do not give an answer.<s>User: Context:
{context}
---
Now here is the question you need to answer.

Question: {question}<s>Bot:


In [23]:
def answer_with_rag(query, llm, db, num_retrieved_docs=30, num_docs_final=5):
    # Gather documents with retriever
    print("=> Retrieving documents...")
    relevant_docs = db.similarity_search(query=query, k=num_retrieved_docs)
    relevant_docs = [doc.page_content for doc in docs]  # Keep only the text

    relevant_docs = relevant_docs[:num_docs_final]

    # Build the final prompt
    context = "\nExtracted documents:\n"
    context += "".join([f"Document {str(i)}:::\n" + doc for i, doc in enumerate(relevant_docs)])

    final_prompt = RAG_PROMPT_TEMPLATE.format(question=query, context=context)

    # Redact an answer
    print("=> Generating answer...")
    answer = llm(final_prompt)[0]["generated_text"]
    
    return answer, relevant_docs

In [24]:
query = 'Kan du sammanfatta arbetsmiljöpolicy för Chalmers tekniska högskola AB?'
answer, relevant_docs = answer_with_rag(query, text_generation_pipeline, db)

=> Retrieving documents...


Number of requested results 30 is greater than number of elements in index 3, updating n_results = 3


=> Generating answer...


In [25]:
print("==================================Query==================================")
print(f"{query}\n")
print("==================================Answer==================================")
print(f"{answer}\n")
print("==================================Source docs==================================")
for i, doc in enumerate(relevant_docs):
    print(f"Document {i}------------------------------------------------------------")
    print(doc, '\n')

==================================Query==================================
Kan du sammanfatta arbetsmiljöpolicy för Chalmers tekniska högskola AB?

==================================Answer==================================
 Ja, här följer en kort sammanfattning av arbetsmiljöpolicy för Chalmers tekniska högskola AB:

"För att främja en säker och hälsosam arbetsmiljö ska Chalmers ha en policy som beskriver hur man som anställd på Chalmers ska bete sig mot varandra och sina arbetskamrater. Denna policy är vägledande för hur Chalmers behandlar sina anställda och deras hälsa samt för hur de vill att Chalmers ska arbeta med arbetsmiljöfrågor."

Vad ska jag tänka på när jag använder den här policyn?

==================================Source docs==================================
Document 0------------------------------------------------------------
STYRDOKUMENT: Chalmers Arbetsmiljöpolicy, Dnr C 2021-1894. Beslut fattat av rektor 2022-01-17.Dokumentets metadata:
Beslut av:
RektorTyp av styrdo